In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [5]:
data = pd.read_csv('fichier.csv')
data.head()

,L,m,theta0_rad,Tc,shape,fluid,dim1,dim2,t_epsilon,dim3
0,1.507481,0.956688,1.321814,0.008680,cylindre,air,0.375333,0.468048,101.702825,NaN
1,1.287720,1.955481,1.405724,0.007229,sphère,air,0.162198,NaN,523.114531,NaN
2,1.877950,4.828315,0.757731,0.002462,sphère,air,0.455437,NaN,879.124420,NaN
3,0.884587,2.358127,1.530066,0.001351,pavé,air,0.262616,0.256551,791.221978,0.392211
4,1.206593,3.870280,1.217885,0.003977,cylindre,air,0.110713,0.053281,1329.036918,NaN


In [6]:
data = data.fillna(0)

In [7]:
# Calcul du volume selon la forme du solide
def calculate_volume(row):
    
    shape = row['shape']
    dim1, dim2, dim3 = row['dim1'], row['dim2'], row['dim3']
    m = row["m"]
    L = row["L"]

    if shape == 'sphère':
        # Volume sphère = (4/3) * π * r³
        # dim1 = rayon
        volume = (4/3) * np.pi * (dim1 ** 3)
        inertie = (2/5) * m * dim1**2 + m * (L + dim1)**2
        surface = 4 * np.pi * (dim1**2)


    elif shape == 'cylindre':
        # Volume cylindre = π * r² * h
        # dim1 = rayon, dim2 = hauteur
        volume = np.pi * (dim1 ** 2) * dim2
        inertie = m * (dim1**2/4 + dim2**2/12) + m * (L + dim1)**2
        surface = np.pi*(dim1**2)
        
    elif shape == 'pavé':
        # Volume pavé = longueur * largeur * hauteur
        # dim1 = longueur, dim2 = largeur, dim3 = hauteur
        volume = dim1 * dim2 * dim3
        inertie = m * (dim1**2 + dim2**2) / 12 + m * (L + dim3/2)**2
        surface = dim2 * dim3
        
    else:
        volume = 0  # Cas par défaut

    return pd.Series([surface, volume, inertie])

In [8]:
# Ajouter la colonne volume
data[['surface', 'volume', 'inertie']] = data.apply(calculate_volume, axis=1)

# Afficher quelques exemples
print("Volume calculé pour chaque forme:")
print(data[['shape', 'dim1', 'dim2', 'dim3', 'volume']].head(10))

Volume calculé pour chaque forme:
      shape      dim1      dim2      dim3    volume
0  cylindre  0.375333  0.468048  0.000000  0.207144
1    sphère  0.162198  0.000000  0.000000  0.017874
2    sphère  0.455437  0.000000  0.000000  0.395706
3      pavé  0.262616  0.256551  0.392211  0.026425
4  cylindre  0.110713  0.053281  0.000000  0.002052
5      pavé  0.034611  0.249150  0.263566  0.002273
6  cylindre  0.082106  0.067608  0.000000  0.001432
7      pavé  0.347276  0.498754  0.048417  0.008386
8    sphère  0.378786  0.000000  0.000000  0.227652
9    sphère  0.232817  0.000000  0.000000  0.052860


In [9]:
FLUID_PROPERTIES = {
    'air':1.225,
    'eau':1000.0,
    'huile': 900.0
}

Cd = {
    'sphère' : 0.47,
    'pavé' : 1.05,
    'cylindre' : 0.82
} 

data['fluid'] = data['fluid'].map(FLUID_PROPERTIES)

In [10]:
keep_cols = [col for col in data.columns if col != 't_epsilon']
keep_cols.append('t_epsilon')
data[keep_cols]

,L,m,theta0_rad,Tc,shape,fluid,dim1,dim2,dim3,surface,volume,inertie,t_epsilon
0,1.507481,0.956688,1.321814,0.008680,cylindre,1.225,0.375333,0.468048,0.000000,0.442570,0.207144,3.442605,101.702825
1,1.287720,1.955481,1.405724,0.007229,sphère,1.225,0.162198,0.000000,0.000000,0.330599,0.017874,4.131510,523.114531
2,1.877950,4.828315,0.757731,0.002462,sphère,1.225,0.455437,0.000000,0.000000,2.606547,0.395706,26.689282,879.124420
3,0.884587,2.358127,1.530066,0.001351,pavé,1.225,0.262616,0.256551,0.392211,0.100622,0.026425,2.780535,791.221978
4,1.206593,3.870280,1.217885,0.003977,cylindre,1.225,0.110713,0.053281,0.000000,0.038508,0.002052,6.728858,1329.036918
...,...,...,...,...,...,...,...,...,...,...,...,...,...
31862,0.254282,0.587679,0.320738,0.003433,sphère,1.225,0.417855,0.000000,0.000000,2.194122,0.305608,0.306539,50.201394
31863,0.385996,1.814723,1.249635,0.004735,cylindre,1000.000,0.023924,0.438739,0.000000,0.001798,0.000789,0.334305,56.601572
31864,1.183854,0.838364,0.782073,0.006325,sphère,1000.000,0.305524,0.000000,0.000000,1.173004,0.119460,1.891002,1.600044
31865,1.017954,0.162440,1.186691,0.001490,sphère,1000.000,0.396998,0.000000,0.000000,1.980549,0.262091,0.335460,1.300036


In [11]:
features = data.drop(['t_epsilon'], axis=1)
target = data['t_epsilon']

In [17]:
features_encoded = pd.get_dummies(features, columns=['shape'], dtype = int)

In [18]:
features_encoded.head()

,L,m,theta0_rad,Tc,fluid,dim1,dim2,dim3,surface,volume,inertie,shape_cylindre,shape_pavé,shape_sphère
0,1.507481,0.956688,1.321814,0.008680,1.225,0.375333,0.468048,0.000000,0.442570,0.207144,3.442605,1,0,0
1,1.287720,1.955481,1.405724,0.007229,1.225,0.162198,0.000000,0.000000,0.330599,0.017874,4.131510,0,0,1
2,1.877950,4.828315,0.757731,0.002462,1.225,0.455437,0.000000,0.000000,2.606547,0.395706,26.689282,0,0,1
3,0.884587,2.358127,1.530066,0.001351,1.225,0.262616,0.256551,0.392211,0.100622,0.026425,2.780535,0,1,0
4,1.206593,3.870280,1.217885,0.003977,1.225,0.110713,0.053281,0.000000,0.038508,0.002052,6.728858,1,0,0


In [13]:
from sklearn.preprocessing import StandardScaler

In [20]:
scaler = StandardScaler()
# cols = ['L', 'm', 'theta0_rad', 'Tc', 'fluid', 'dim1', 'dim2', 'dim3', 'surface', 'volume', 'inertie']
features_encoded = scaler.fit_transform(features_encoded)

In [21]:
X_train, X_test, y_train, y_test = train_test_split(features_encoded, target, test_size = 0.2, train_size = 0.8, random_state = 42)

In [22]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, classification_report

In [23]:
# 5. MLP NEURAL NETWORK
print("5. Entraînement MLP...")
results = {}
mlp = MLPRegressor(
    hidden_layer_sizes=(200, 150, 100), 
    max_iter = 500,
    random_state=42,
    alpha = 0.0001,
    )

mlp.fit(X_train, y_train)
y_pred_mlp = mlp.predict(X_test)
results['MLP'] = {
    'RMSE': np.sqrt(mean_squared_error(y_test, y_pred_mlp)),
    'MAE': mean_absolute_error(y_test, y_pred_mlp),
    'R²': r2_score(y_test, y_pred_mlp)
}

print(f"RMSE : {results["MLP"]["RMSE"]}")
print(f"MAE : {results["MLP"]["MAE"]}")
print(f"R2 : {results["MLP"]["R²"]}")

5. Entraînement MLP...
RMSE : 40.343458715456705
MAE : 12.221618896605037
R2 : 0.991494757126867


Enregistrement du modèle

In [24]:
from sklearn.datasets import make_regression
import joblib

In [25]:
# --- Sauvegarde du modèle ---
path = 'C:/Users/EEIA/Documents/pendule/web_interface/'
joblib.dump(mlp, path + 'mlp_model_4_1.pkl')
joblib.dump(scaler, path + 'scaler_4_1.pkl')
print(f"Modèle enregistré sous: {path + 'mlp_model_4_1.pkl'}")

Modèle enregistré sous: C:/Users/EEIA/Documents/pendule/web_interface/mlp_model_4_1.pkl
